In [135]:
import pandas as pd
import numpy as np
import ast
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.feature_extraction.text import TfidfVectorizer
# Dataset : https://www.kaggle.com/datasets/fmpugliese/steam-all-games-data

# IMDB TOP 250 RATING : https://help.imdb.com/article/imdb/track-movies-tv/ratings-faq/G67Y87TFYYP6TWAV#

In [136]:
# Data Collection & Feature Engineering
# Ce df va principalement servir au calcul de la variable y (à l'exception de initialprice)
game_data = pd.read_csv('dataset/all_data.csv', sep=',')
try:
    # Ce df contient nos principales features (raw)
    steam_app_data = pd.read_csv('dataset/steam_app_data.csv', sep=',', engine='python', on_bad_lines='skip')
except pd.errors.ParserError as e:
    print(f"Critical ParserError encountered: {e}. Even with a robust engine and skipping bad lines, the file cannot be read automatically. Please inspect 'steam_app_data.csv' for severe formatting issues around the reported line number.")
    raise # Re-raise the exception to stop execution and indicate failure

#df = pd.merge(game_data, steam_app_data, on='app_id', how='inner')
print("game_data columns", game_data.columns)
print("steam_app_data columns", steam_app_data.columns)

# reprendre lecture CSV comme pour jester ratings (td6) ?

game_data columns Index(['Unnamed: 0', 'appid', 'name', 'developer', 'publisher', 'score_rank',
       'positive', 'negative', 'userscore', 'owners', 'average_forever',
       'average_2weeks', 'median_forever', 'median_2weeks', 'price',
       'initialprice', 'discount', 'ccu'],
      dtype='object')
steam_app_data columns Index(['type', 'name', 'steam_appid', 'required_age', 'is_free',
       'controller_support', 'dlc', 'detailed_description', 'about_the_game',
       'short_description', 'fullgame', 'supported_languages', 'header_image',
       'website', 'pc_requirements', 'mac_requirements', 'linux_requirements',
       'legal_notice', 'drm_notice', 'ext_user_account_notice', 'developers',
       'publishers', 'demos', 'price_overview', 'packages', 'package_groups',
       'platforms', 'metacritic', 'reviews', 'categories', 'genres',
       'screenshots', 'movies', 'recommendations', 'achievements',
       'release_date', 'support_info', 'background', 'content_descriptors'],
    

In [137]:
# Removing features we will not use for our model, and renaming the id variable for the join that we will do later
feature = [ "appid", "initialprice", "is_free", "genres", "categories", "required_age", "short_description", "name", "supported_languages", "controller_support", "platforms" ]
rename = { "steam_appid": "appid" }
steam_app_data = steam_app_data.rename(columns=rename)
for column in steam_app_data.columns:
    if column not in feature:
        steam_app_data.drop(column, axis=1, inplace=True)

print("steam_app_data New columns", steam_app_data.columns)
print("First row", steam_app_data.iloc[0])

steam_app_data New columns Index(['name', 'appid', 'required_age', 'is_free', 'controller_support',
       'short_description', 'supported_languages', 'platforms', 'categories',
       'genres'],
      dtype='object')
First row name                                                      Counter-Strike
appid                                                                 10
required_age                                                         0.0
is_free                                                            False
controller_support                                                   NaN
short_description      Play the world's number 1 online action game. ...
supported_languages    English<strong>*</strong>, French<strong>*</st...
platforms                  {'windows': True, 'mac': True, 'linux': True}
categories             [{'id': 1, 'description': 'Multi-player'}, {'i...
genres                            [{'id': '1', 'description': 'Action'}]
Name: 0, dtype: object


In [138]:
score_variables = [ 'appid', 'positive', 'negative', 'initialprice' ] # initialprice not used for computing score but we must keep it for the merge because we need it as a feature
for column in game_data.columns:
    if column not in score_variables:
        game_data.drop(column, axis=1, inplace=True)
# On utilise reviews plutôt que owners car c'est directement lié au ratio
game_data['total_reviews'] = game_data['positive'] + game_data['negative']
game_data['positive_ratio'] = game_data['positive'] / game_data['total_reviews']

# C : La moyenne globale de tous les jeux (La "baseline")
C = game_data['positive_ratio'].mean()

# m : Le nombre de reviews minimum pour "faire confiance" à la note
# On peut prendre le quantile 50% (médiane) ou 70% pour être exigeant
m = game_data['total_reviews'].quantile(0.75)
print('Minimal number of reviews for positive_ratio to be "trusted":', m)

# La fonction de pondération IMDb TOP 250 (source : https://help.imdb.com/article/imdb/track-movies-tv/ratings-faq/G67Y87TFYYP6TWAV#)
def weighted_rating(x, m=m, C=C):
    v = x['total_reviews']
    R = x['positive_ratio']
    # Formule mathématique
    return (v/(v+m) * R) + (m/(v+m) * C)

game_data['score_pondere'] = game_data.apply(weighted_rating, axis=1)

# Création de ta Target Binaire (Y)
# On définit un "Hit" comme faisant partie du Top 25% des meilleurs scores pondérés
hit_threshold = game_data['score_pondere'].quantile(0.80)
print("Score threshold to be considered a hit game:", hit_threshold)
game_data['is_hit'] = game_data['score_pondere'].apply(lambda x: 1 if x > hit_threshold else 0)
print(game_data.columns)
for column in game_data.columns:
    if column not in ['appid', 'initialprice', 'is_hit']:
        game_data.drop(column, axis=1, inplace=True)

print("game_data New columns", game_data.columns)
print("First row", game_data.iloc[0])

Minimal number of reviews for positive_ratio to be "trusted": 128.0
Score threshold to be considered a hit game: 0.7931399045991935
Index(['appid', 'positive', 'negative', 'initialprice', 'total_reviews',
       'positive_ratio', 'score_pondere', 'is_hit'],
      dtype='object')
game_data New columns Index(['appid', 'initialprice', 'is_hit'], dtype='object')
First row appid            10.0
initialprice    999.0
is_hit            1.0
Name: 0, dtype: float64


In [139]:
# HANDLING DUPLICATES
print(steam_app_data.shape, "Number of duplicates:", len(steam_app_data[steam_app_data.duplicated(subset='appid', keep=False)])   )
print("Keeping only the last row for each duplicate appid value...")
steam_app_data.drop_duplicates(subset='appid', keep='last', inplace=True) # Noticed in the dataset always last ones were in english
print(steam_app_data.shape, "Number of duplicates now:", len(steam_app_data[steam_app_data.duplicated(subset='appid', keep=False)])   )

print(game_data.shape, "Number of duplicates:", len(game_data[game_data.duplicated(subset='appid', keep=False)])   )
print("Keeping only the last row for each duplicate appid value...")
game_data.drop_duplicates(subset='appid', keep='last', inplace=True) # Noticed in the dataset always last ones were in english
print(game_data.shape, "Number of duplicates now:", len(game_data[game_data.duplicated(subset='appid', keep=False)])   )

(86538, 10) Number of duplicates: 8152
Keeping only the last row for each duplicate appid value...
(82458, 10) Number of duplicates now: 0
(86538, 3) Number of duplicates: 8076
Keeping only the last row for each duplicate appid value...
(82500, 3) Number of duplicates now: 0


In [140]:
print(game_data.shape)
print(steam_app_data.shape)

dataset = pd.merge(steam_app_data, game_data, on='appid', how='inner')
print("Dataset shape", dataset.shape)
print("Number of duplicate appids", len(dataset[dataset['appid'].duplicated()]['appid'].tolist()))

(82500, 3)
(82458, 10)
Dataset shape (82413, 12)
Number of duplicate appids 0


In [141]:
#dataset = dataset.dropna()
#print("Dataset shape after removing all null values", dataset.shape)

In [142]:
print(dataset[dataset.isnull().any(axis=1)].iloc[0])

name                                                      Counter-Strike
appid                                                                 10
required_age                                                         0.0
is_free                                                            False
controller_support                                                   NaN
short_description      Play the world's number 1 online action game. ...
supported_languages    English<strong>*</strong>, French<strong>*</st...
platforms                  {'windows': True, 'mac': True, 'linux': True}
categories             [{'id': 1, 'description': 'Multi-player'}, {'i...
genres                            [{'id': '1', 'description': 'Action'}]
initialprice                                                       999.0
is_hit                                                                 1
Name: 0, dtype: object


DATA ENGINEERING OF EACH FEATURE AND HANDLING OF NULL VALUES

In [143]:
# name
# remove all rows where name is null (will do the NLP on this column later)
dataset = dataset.dropna(subset=['name'])

In [144]:
# is_free
# By inspecting randomly a subset of rows where is_free is null,
# I noticed it's all irrelevant rows that have full null values (apart from name and is_hit),
# better of just removing them
dataset = dataset.dropna(subset=['is_free'])
print(dataset.shape)
print(dataset['is_free'].unique())
print("Converting all False and True to int values 0 and 1")
dataset['is_free'] = dataset['is_free'].astype(int)
print(dataset.shape)
print(dataset['is_free'].unique())
#print(dataset[dataset['is_free'] == 0])

(82186, 12)
[False True]
Converting all False and True to int values 0 and 1
(82186, 12)
[0 1]


In [145]:
#controller_support
print(dataset['controller_support'].unique())
print(dataset.shape)
# Replacing null by 0
dataset["controller_support"] = dataset["controller_support"].fillna(0)
# Replacing 'full' by 1
dataset["controller_support"] = dataset["controller_support"].replace('full', 1)
print(dataset['controller_support'].unique())
print(dataset.shape)

[nan 'full']
(82186, 12)
[0 1]
(82186, 12)


In [146]:
#required_age
print(dataset['required_age'].unique())
# Noticing no NaN values, and floats have not decimals, so converting to int
dataset['required_age'] = dataset['required_age'].astype(int)
print(dataset['required_age'].dtype)

[ 0. 12. 17. 14. 18. 16. 10. 15. 13. 11.  3.  7.  6.  1.  5.  8. 19.]
int32


In [147]:
#initialprice
print(dataset['initialprice'].describe())
dataset['initialprice'] = dataset['initialprice'] / 100
print(dataset['initialprice'].describe())

count    82186.000000
mean       830.387110
std       1355.268013
min          0.000000
25%        199.000000
50%        499.000000
75%        999.000000
max      99998.000000
Name: initialprice, dtype: float64
count    82186.000000
mean         8.303871
std         13.552680
min          0.000000
25%          1.990000
50%          4.990000
75%          9.990000
max        999.980000
Name: initialprice, dtype: float64


In [148]:
# platforms
print(dataset['platforms'].unique()) # No null values
# Function that changes the string into a dict
def extract_platforms(text_data):
    try:
        data_dict = ast.literal_eval(text_data)
        return data_dict # Returns {'windows': True, ...}
    except:
        return {'windows': False, 'mac': False, 'linux': False}

# Applying the string to dict function then using the pandas function that transforms dict keys into columns
platforms_df = dataset['platforms'].apply(extract_platforms).apply(pd.Series)
# Concat new columns to our dataset
dataset = pd.concat([dataset, platforms_df], axis=1)
# Dropping old 'platforms' column not useful anymore
dataset.drop('platforms', axis=1, inplace=True)
print(dataset.columns)
print(dataset['windows'].unique())
print(dataset['mac'].unique())
print(dataset['linux'].unique())

["{'windows': True, 'mac': True, 'linux': True}"
 "{'windows': True, 'mac': False, 'linux': True}"
 "{'windows': True, 'mac': False, 'linux': False}"
 "{'windows': True, 'mac': True, 'linux': False}"
 "{'windows': False, 'mac': True, 'linux': False}"
 "{'windows': False, 'mac': False, 'linux': True}"
 "{'windows': False, 'mac': True, 'linux': True}"]
Index(['name', 'appid', 'required_age', 'is_free', 'controller_support',
       'short_description', 'supported_languages', 'categories', 'genres',
       'initialprice', 'is_hit', 'windows', 'mac', 'linux'],
      dtype='object')
[ True False]
[ True False]
[ True False]


In [149]:
# genres

# Dictionnaire de consolidation (Basé sur ta liste)
genre_map = {
    # --- ACTION ---
    'Acción': 'Action', 'Action': 'Action', 'Akcja': 'Action', 'Akční': 'Action',
    'Aksi': 'Action', 'Azione': 'Action', 'Ação': 'Action', 'Hành động': 'Action',
    'Бойовики': 'Action', 'Экшены': 'Action', '动作': 'Action', '動作': 'Action',
    'Δράση': 'Action', 'Akció': 'Action',

    # --- ADVENTURE ---
    'Abenteuer': 'Adventure', 'Adventure': 'Adventure', 'Aventura': 'Adventure',
    'Aventure': 'Adventure', 'Avventura': 'Adventure', 'Dobrodružné': 'Adventure',
    'Eventyr': 'Adventure', 'Phiêu lưu': 'Adventure', 'Przygodowe': 'Adventure',
    'Пригоди': 'Adventure', 'Приключенческие игры': 'Adventure', 'アドベンチャー': 'Adventure',
    '冒险': 'Adventure', '冒險': 'Adventure',

    # --- RPG ---
    'GDR': 'RPG', 'Nhập vai (RPG)': 'RPG', 'RPG': 'RPG', 'Rol': 'RPG', 'Rollenspiel': 'RPG',
    'Role Playing': 'RPG', 'Ролевые игры': 'RPG', '角色扮演': 'RPG', 'J-RPG': 'RPG',

    # --- STRATEGY ---
    'Estrategia': 'Strategy', 'Estratégia': 'Strategy', 'Strategia': 'Strategy',
    'Strategie': 'Strategy', 'Strategy': 'Strategy', 'Stratégie': 'Strategy',
    'Стратегии': 'Strategy', '策略': 'Strategy',

    # --- SIMULATION ---
    'Simuladores': 'Simulation', 'Simulation': 'Simulation', 'Simulationen': 'Simulation',
    'Simulátory': 'Simulation', 'Symulacje': 'Simulation', 'Симулятори': 'Simulation',
    'Симуляторы': 'Simulation', '模拟': 'Simulation', '模擬': 'Simulation', 'Simulação': 'Simulation',

    # --- RACING ---
    'Carreras': 'Racing', 'Course automobile': 'Racing', 'Racing': 'Racing',
    'Versenyzés': 'Racing', 'Wyścigowe': 'Racing', 'Гонки': 'Racing', '競速': 'Racing',

    # --- SPORTS ---
    'Deportes': 'Sports', 'Sportowe': 'Sports', 'Sports': 'Sports', 'Спорт': 'Sports', '運動': 'Sports',

    # --- CASUAL ---
    'Casual': 'Casual', 'Gelegenheitsspiele': 'Casual', 'Occasionnel': 'Casual',
    'Rekreacyjne': 'Casual', 'Казуальные игры': 'Casual', 'Казуальні ігри': 'Casual',
    '休闲': 'Casual', '休閒': 'Casual', 'カジュアル': 'Casual',

    # --- INDIE ---
    'Indie': 'Indie', 'Indépendant': 'Indie', 'Nezávislé': 'Indie', 'Niezależne': 'Indie',
    'Інді': 'Indie', 'Инди': 'Indie', '独立': 'Indie', '獨立製作': 'Indie',

    # --- MMO ---
    'MMO': 'Massively Multiplayer', 'Masivně multiplayerové': 'Massively Multiplayer',
    'Massively Multiplayer': 'Massively Multiplayer', 'Massivement multijoueur': 'Massively Multiplayer',
    'Multijogador Massivo Online (MMO)': 'Massively Multiplayer', 'Multijugador masivo': 'Massively Multiplayer',
    'Sokszereplős többjátékos': 'Massively Multiplayer', 'Многопользовательские игры': 'Massively Multiplayer',

    # --- EARLY ACCESS ---
    'Early Access': 'Early Access', 'Acceso anticipado': 'Early Access', 'Accès anticipé': 'Early Access',
    'Acesso Antecipado': 'Early Access', 'Wczesny dostęp': 'Early Access', 'Ранний доступ': 'Early Access', 'Вільний доступ': 'Early Access', '抢先体验': 'Early Access',

    # --- FREE to PLAY ---
    'Free to Play': 'Free to Play', 'Free To Play': 'Free to Play', 'Free to play': 'Free to Play',
    'Gratuitos para Jogar': 'Free to Play', 'Бесплатные': 'Free to Play', '免费开玩': 'Free to Play', '無料プレイ': 'Free to Play',

    '实用工具': 'Software'  # Translation is 'Practical Tools', not sure which genre exactly that is so just putting it in 'Software'
}

# Function that transforms strings that contain {id: int, description: genre}
# into a list of genres from the name of the genre in the variable description
# AND translates all genres that we have in different languages into one language
def extract_and_map_genres(text_data):
    if pd.isna(text_data):
        return []
    try:
        data_list = ast.literal_eval(text_data)
        extracted = []
        for item in data_list:
            raw_genre = item['description']
            # if in our translation dict translate, else ignore
            if raw_genre in genre_map:
                extracted.append(genre_map[raw_genre])
            else:
              if raw_genre in ['Accounting', 'Animation & Modeling', 'Audio Production', 'Design & Illustration', 'Education', 'Photo Editing', 'Software Training', 'Utilities', 'Video Production', 'Web Publishing']:
                 #Optionnal : Regrouping all non-games under "Software"
                 extracted.append('Software')
              extracted.append(raw_genre)
        # to avoid duplciates we use set()
        return list(set(extracted))

    except (ValueError, SyntaxError):
        return []

# temporary column
dataset['genres_list'] = dataset['genres'].apply(extract_and_map_genres)
# using MultiLabelBinarizer to transform genres list into a 0 and 1 matrix
mlb = MultiLabelBinarizer()
genres_encoded = mlb.fit_transform(dataset['genres_list'])

# creating new dataframe with the new columns, each name starting with "genre_"
genres_df = pd.DataFrame(
    genres_encoded,
    columns=[f"genre_{cls}" for cls in mlb.classes_],
    index=dataset.index
)
# concat new columns to our dataset
dataset = pd.concat([dataset, genres_df], axis=1)

# dropping old genre columns
dataset = dataset.drop(columns=['genres', 'genres_list'])
# show new genre columns (thanks to translating grouping, only 30 compared to 117)
print(dataset.filter(like='genre_').columns.tolist())

['genre_Accounting', 'genre_Action', 'genre_Adventure', 'genre_Animation & Modeling', 'genre_Audio Production', 'genre_Casual', 'genre_Design & Illustration', 'genre_Early Access', 'genre_Education', 'genre_Free to Play', 'genre_Game Development', 'genre_Gore', 'genre_Indie', 'genre_Massively Multiplayer', 'genre_Movie', 'genre_Nudity', 'genre_Photo Editing', 'genre_RPG', 'genre_Racing', 'genre_Sexual Content', 'genre_Short', 'genre_Simulation', 'genre_Software', 'genre_Software Training', 'genre_Sports', 'genre_Strategy', 'genre_Utilities', 'genre_Video Production', 'genre_Violent', 'genre_Web Publishing']


In [150]:
print(dataset.iloc[0]['categories'])

[{'id': 1, 'description': 'Multi-player'}, {'id': 49, 'description': 'PvP'}, {'id': 36, 'description': 'Online PvP'}, {'id': 37, 'description': 'Shared/Split Screen PvP'}, {'id': 66, 'description': 'Color Alternatives'}, {'id': 68, 'description': 'Custom Volume Controls'}, {'id': 75, 'description': 'Keyboard Only Option'}, {'id': 69, 'description': 'Stereo Sound'}, {'id': 8, 'description': 'Valve Anti-Cheat enabled'}, {'id': 62, 'description': 'Family Sharing'}]


In [151]:
# categories
#print(dataset['categories'].unique().tolist())
import pandas as pd
import ast
from sklearn.preprocessing import MultiLabelBinarizer

# Mapping function that translates all categories to english
def get_english_category(text):
    if not isinstance(text, str):
        return None

    t = text.lower()

    # --- 1. MODES DE JEU (Joueurs) ---
    # Ajouts : Vietnamien (chơi đơn), Grec (ένας παίκτης)
    if any(x in t for x in ['single-player', 'single player', 'un jugador', 'um jogador', 'einzelspieler', 'solo', 'giocatore singolo', 'jednoosobowa', 'enkeltspiller', 'egyjátékos', 'pemain tunggal', 'režim pro jednoho', 'одного игрока', 'однокористувацька', '单人', '單人', 'シングルプレイヤー', 'chơi đơn', 'ένας παίκτης']):
        return 'Single-player'

    if any(x in t for x in ['mmo', 'massively multiplayer', 'massif', 'masivo', 'massivo', 'sokszereplős', 'многопользовательские', 'masivně']):
        return 'MMO'

    if any(x in t for x in ['multi-player', 'multiplayer', 'mehrspieler', 'multijoueur', 'multijugador', 'multijogador', 'multigiocatore', 'wieloosobowa', 'multipemain', 'nhiều người chơi', 'többjátékos', 'režim pro více', 'нескольких игроков', 'багатокористувацька', '多人', 'багатокористувацька']):
        return 'Multi-player'

    if any(x in t for x in ['co-op', 'coop', 'ko-op', 'koop', 'coopération', 'cooperativo', 'kooperacja', 'phối hợp', '合作', 'кооператив']):
        if any(sub in t for sub in ['online', 'ligne', 'red', 'sieciowa', 'trực tuyến', '在线', '線上', 'мережі']):
            return 'Online Co-op'
        if any(sub in t for sub in ['lan', 'local', 'lauk', '局域网']):
            return 'LAN Co-op'
        return 'Co-op'

    if any(x in t for x in ['pvp', 'jcj', 'jxj', 'player vs', 'graвець', 'игрок против', '玩家对战', '玩家對戰']):
        if any(sub in t for sub in ['online', 'ligne', 'red', 'sieciowe', 'мережі']):
            return 'Online PvP'
        if 'lan' in t:
            return 'LAN PvP'
        return 'PvP'

    if any(x in t for x in ['shared/split', 'split screen', 'écran partagé', 'pantalla partida', 'geteilt', 'dzielony', '同屏', '分割', 'разделённый']):
        return 'Shared/Split Screen'

    if any(x in t for x in ['cross-platform', 'cross platform', 'multiplateforme', 'multiplataforma', 'wieloplatformowa', '跨平台', 'кросс-платформенный', 'міжплатформна']):
        return 'Cross-Platform Multiplayer'

    # --- 2. FEATURES TECHNIQUES ---
    # Ajouts : Norvégien (støtte), Hongrois (támogatás), Grec (υποστήριξη), Cyrillique étendu
    if any(x in t for x in ['controller', 'contrôleurs', 'control', 'mando', 'comando', 'tay cầm', 'kontroler', 'gamepad', 'joypad', 'shoubing', 'ovladačů', '控制器', 'コントローラ', 'støtte', 'támogatás', 'υποστήριξη', 'χειριστηρίου', 'підтримка']):
        if any(sub in t for sub in ['full', 'complète', 'total', 'pełna', 'plná', 'полная', '完全', 'πλήρης']):
            return 'Full Controller Support'
        return 'Partial Controller Support'

    if any(x in t for x in ['vr', 'virtual reality', 'réalité virtuelle', 'вр']):
        if any(sub in t for sub in ['only', 'liše', 'nur', 'seul', 'только', 'лише']):
            return 'VR Only'
        return 'VR Support'

    if 'hdr' in t:
        return 'HDR Available'

    # NOUVEAU : Audio Tech
    if any(x in t for x in ['stereo', 'stéréo', 'estéreo', '立体声', '立體聲']):
        return 'Stereo Sound'

    if any(x in t for x in ['surround', 'przestrzenny', 'prostorový', 'envolvente', 'объёмный', '环绕', '環繞']):
        return 'Surround Sound'

    # --- 3. STEAM FEATURES ---
    # Ajouts : Grec, Vietnamien, Norvégien, etc.
    if any(x in t for x in ['achievements', 'succès', 'logros', 'conquistas', 'osiągnięcia', 'thành tựu', 'pencapaian', 'akrre', 'errungenschaften', 'достижения', 'досягнення', '成就', '実績', 'achievement', 'επιτεύγματα', 'proezas', 'prestasjoner']):
        return 'Steam Achievements'

    if any(x in t for x in ['trading cards', 'cartes à échanger', 'cromos', 'cartas', 'karty', 'thẻ', 'sammelkarten', 'карточки', 'картки', '卡牌', '卡片', 'samlekort', 'collezionabili', '交換卡片']):
        return 'Steam Trading Cards'

    if any(x in t for x in ['cloud', 'nuvem', 'wolke', 'chmura', 'облако', '云', '雲端', 'クラウド']):
        return 'Steam Cloud'

    if any(x in t for x in ['workshop', 'oficina', 'warsztat', 'atelier', 'werkstatt', 'мастерская', 'майстерня', '工坊']):
        return 'Steam Workshop'

    if any(x in t for x in ['leaderboards', 'classements', 'classificações', 'bestenlisten', 'rankingi', 'tabelas', 'tablas', '排行榜', 'classifiche', 'таблицы']):
        return 'Steam Leaderboards'

    if any(x in t for x in ['family', 'famille', 'familia', 'rodzin', 'családi', 'chia sẻ', 'семейный', 'сімейна', '家庭', '亲友', '親友', 'familiedeling', 'udostępnianie', 'κοινή χρήση', 'ファミリーシェアリング']):
        return 'Family Sharing'

    if any(x in t for x in ['remote play', 'remote\xa0play', 'yuancheng', '遠端', '远程']):
        return 'Remote Play'

    if any(x in t for x in ['turn notifications', '回合通知']):
        return 'Steam Turn Notifications'

    if any(x in t for x in ['timeline', '時間軸', '时间轴', 'oś czasu', 'linha do tempo', 'línea de tiempo']):
        return 'Steam Timeline'

    # --- 4. ACCESSIBILITÉ & OPTIONS (Nouveau bloc important) ---

    # Couleurs
    if any(x in t for x in ['color', 'couleurs', 'cores', 'barwy', 'nastavení barev', 'цвета', '颜色', '顏色']):
        return 'Color Alternatives'

    # Difficulté
    if any(x in t for x in ['difficulty', 'difficulté', 'dificuldade', 'dificultad', 'schwierigkeitsgrad', 'poziom trudności', 'сложность', 'skladnosť', '難度']):
        return 'Adjustable Difficulty'

    # Taille du texte
    if any(x in t for x in ['text size', 'taille', 'tamanho', 'rozmiar', 'размер', '文字大小']):
        return 'Adjustable Text Size'

    # Audio / Volume
    if any(x in t for x in ['volume', 'lautstärke', 'głośności', 'hlasitosti', '音量']):
        return 'Custom Volume Controls'

    # Sauvegarde
    if any(x in t for x in ['save', 'sauvegarde', 'guardar', 'salvamento', 'zapis', 'сохранение', '保存', '儲存']):
        return 'Save Anytime'

    # QTE / Input
    if any(x in t for x in ['timed input', 'qte', 'reação', 'temps imparti', 'eventos', 'zeitgesteuerte', 'reaktion', '反应', '反應', 'saisie']):
        return 'Playable without Timed Input'

    # Clavier / Souris seulement
    if any(x in t for x in ['keyboard only', 'clavier uniquement', 'nur tastatur', 'tastatur', '鍵盤', '键盘']):
        return 'Keyboard Only Option'
    if any(x in t for x in ['mouse only', 'souris uniquement', 'tylko mysz', 'nur maus', '滑鼠', '鼠标']):
        return 'Mouse Only Option'

    # --- 5. DIVERS ---
    if any(x in t for x in ['in-app', 'achats', 'compras', 'zakupy', 'kauf', 'vásárlások', 'mua hàng', 'покупки', 'придбання', '购买', '購買']):
        return 'In-App Purchases'

    if any(x in t for x in ['stat', 'estat', 'statis', '统计数据']):
        return 'Stats'

    if any(x in t for x in ['editor', 'éditeur', 'redaktor', 'edytor', 'редактор', '编辑器']):
        return 'Includes Level Editor'

    if any(x in t for x in ['sdk', 'source']):
        return 'Includes Source SDK'

    if 'demo' in t:
        return 'Game Demo'

    if any(x in t for x in ['caption', 'subtitle', 'sous-titr', 'subtít', 'napisy', 'legendas', 'teksting', 'zimaku', 'субтитры', '字幕', 'comentario', 'commentary', 'comentários']):
        return 'Captions/Subtitles'

    if any(x in t for x in ['anti-cheat', 'antitrapaça', 'anti-triche', 'antitrampas', 'anti cheat', 'античит', '反作弊']):
        return 'Valve Anti-Cheat'

    if any(x in t for x in ['speech-to-text', 'text-to-speech']):
        return 'Text/Speech Conversion'

    # Mods
    if 'mods' in t:
        return 'Mods'

    return text

# /!\ To decrease noise, we remove all categories that have less than 1000 occurences (only impacts 1% of dataset)
def filter_less_relevant_categories(categorie):
  if categorie in ['Stereo Sound', 'Camera Comfort', '视角舒适度', '鏡頭舒適度', 'Adjustable Difficulty', '可调整难度', 'Save Anytime', 'Mouse Only Option', 'Keyboard Only Option', 'Color Alternatives', 'Adjustable Text Size', 'Surround Sound', 'Touch Only Option', 'HDR Available', 'Valve Anti-Cheat', 'Steam Turn Notifications', 'Narrated Game Menus', 'Includes Source SDK', 'Steam Timeline', 'Полная поддержка контроллеров', 'Familienbibliothek', 'Частичная поддержка контроллеров', 'Text/Speech Conversion', 'Compartilhamento em família', 'Custom Volume Controls', 'Tarjetas de Steam', 'Game Demo', 'Управление только клавиатурой', 'Dostosowywanie kamery', 'Стереозвук', 'Расширенное управление громкостью', 'Grywalne bez określonych w czasie działań', 'Статистика', 'Управление только мышью', 'Conforto de câmera', 'Käufe im Spiel', 'Гравець проти гравця в мережі', 'Настройки субтитров', 'Настройки движения камеры', 'Гравець проти гравця', 'Comodidad de la cámara', 'Opcje napisów', 'Pembelian dalam Aplikasi', 'З субтитрами', 'Partilha de Biblioteca', 'Поддержка отслеживания контроллеров', 'Herní obchod', 'Untertitel verfügbar', 'Sdílení v rodině', 'Озвученные игровые меню', 'Caméra et confort de vue', 'Narração de menus', '朗讀遊戲選單']:
    return None
  else:
    return categorie

# Function that transforms strings that contain {id: int, description: genre} into a list of categories from the name of the genre in the variable description
def extract_categorie_names(text_data):
    if pd.isna(text_data):
        return []
    try:
        data_list = ast.literal_eval(text_data)
        extracted = []
        for item in data_list:
          if filter_less_relevant_categories(get_english_category(item['description'])):
            extracted.append(get_english_category(item['description']))
        # to avoid duplciates we use set()
        return list(set(extracted))
    except (ValueError, SyntaxError):
        return []

# temporary column
dataset['categories_list'] = dataset['categories'].apply(extract_categorie_names)

# using MultiLabelBinarizer to transform categories list into a 0 and 1 matrix
mlb = MultiLabelBinarizer()
categories_encoded = mlb.fit_transform(dataset['categories_list'])

# creating new dataframe with the new columns, each name starting with "categorie_"
categories_df = pd.DataFrame(
    categories_encoded,
    columns=[f"categorie_{cls}" for cls in mlb.classes_],
    index=dataset.index
)

# oncat new columns to our dataset
dataset = pd.concat([dataset, categories_df], axis=1)

# dropping old genre columns
dataset = dataset.drop(columns=['categories', 'categories_list'])
print([ categorie.replace("categorie_", "") for categorie in dataset.filter(like='categorie_').columns.tolist() ])
print(len(dataset.filter(like='categorie_').columns.tolist()))


['Captions/Subtitles', 'Co-op', 'Cross-Platform Multiplayer', 'Family Sharing', 'Full Controller Support', 'In-App Purchases', 'Includes Level Editor', 'LAN Co-op', 'LAN PvP', 'MMO', 'Mods', 'Multi-player', 'Online Co-op', 'Online PvP', 'Partial Controller Support', 'Playable without Timed Input', 'PvP', 'Remote Play', 'Shared/Split Screen', 'Single-player', 'Stats', 'Steam Achievements', 'Steam Cloud', 'Steam Leaderboards', 'Steam Trading Cards', 'Steam Workshop', 'Trading Card Steam', 'VR Only', 'VR Support']
29


In [152]:
#all_cats = dataset['categories_list'].explode()
#print(all_cats.value_counts())
#print (all_cats.value_counts()[all_cats.value_counts() < 1000].keys().tolist())
#print (len(all_cats.value_counts()[all_cats.value_counts() < 1000].keys().tolist()))

In [153]:
# supported_languages
dataset = dataset.dropna(subset=['supported_languages'])

In [154]:
# supported_languages
print( dataset['supported_languages'].iloc[0] )

print( len(dataset[ dataset['supported_languages'].str.contains("Portuguese - Portugal", na=False) ]) )
print( len(dataset[ dataset['supported_languages'].str.contains("Portuguese - Brazil", na=False) ]) )
print( len(dataset[ dataset['supported_languages'].str.contains("Portuguese - Portugal", na=False) & dataset['supported_languages'].str.contains("Portuguese - Brazil", na=False) ]) )

print( len(dataset[ dataset['supported_languages'].str.contains("Spanish - Spain", na=False) ]) )
print( len(dataset[ dataset['supported_languages'].str.contains("Spanish - Latin America", na=False) ]) )
print( len(dataset[ dataset['supported_languages'].str.contains("Spanish - Latin America", na=False) & dataset['supported_languages'].str.contains("Spanish - Latin America", na=False) ]) )

English<strong>*</strong>, French<strong>*</strong>, German<strong>*</strong>, Italian<strong>*</strong>, Spanish - Spain<strong>*</strong>, Simplified Chinese<strong>*</strong>, Traditional Chinese<strong>*</strong>, Korean<strong>*</strong><br><strong>*</strong>languages with full audio support
6943
11848
3919
19890
5647
5647


In [155]:
# UTILITIES

lookup_langues = {
    # --- English ---
    'English': 'English',
    'Inglés': 'English',
    'Inglês': 'English',
    'Angielski': 'English',
    'английский': 'English',
    'Inglese': 'English',
    'англійська': 'English',
    '英语': 'English',
    'Inglésidiomas': 'English',
    'Englisch': 'English',
    'Angol': 'English',
    'Anglais': 'English',
    'Bhs. Inggris': 'English',
    'Αγγλικά': 'English',
    'Engelsk': 'English',
    '英語': 'English',
    'Tiếng Anh': 'English',
    '英文': 'English',
    'Angielskijęzyki z pełnym udźwiękowieniem': 'English',
    'Angličtina': 'English',

    # --- French ---
    'French': 'French',
    'Francés': 'French',
    'Francês': 'French',
    'Francuski': 'French',
    '#lang_français': 'French',
    'французский': 'French',
    'Francese': 'French',
    'французька': 'French',
    '法语': 'French',
    'Französisch': 'French',
    'Francia': 'French',
    'Français': 'French',
    'Bhs. Prancis': 'French',
    'Γαλλικά': 'French',
    'Fransk': 'French',
    'Tiếng Pháp': 'French',
    '法文': 'French',
    'Francouzština': 'French',

    # --- German ---
    'German': 'German',
    'Alemán': 'German',
    'Alemão': 'German',
    'Niemiecki': 'German',
    'немецкий': 'German',
    'Tedesco': 'German',
    'German;': 'German',
    'німецька': 'German',
    '德语': 'German',
    'Deutsch': 'German',
    'Német': 'German',
    'Allemand': 'German',
    'Bhs. Jerman': 'German',
    'Γερμανικά': 'German',
    'Tysk': 'German',
    'Tiếng Đức': 'German',
    '德文': 'German',
    'Němčina': 'German',

    # --- Spanish ---
    'Spanish': 'Spanish',
    'Spanish - Spain': 'Spanish',
    'Español - España': 'Spanish',
    'Español - Latinoamérica': 'Spanish',
    'Spanish - Latin America': 'Spanish',
    'Espanhol (Espanha)': 'Spanish',
    'Espanhol (América Latina)': 'Spanish',
    'Hiszpański': 'Spanish',
    'Hiszpański latynoamerykański': 'Spanish',
    'испанский — Испания': 'Spanish',
    'Spagnolo - Spagna': 'Spanish',
    'іспанська (Іспанія)': 'Spanish',
    '西班牙语 - 西班牙': 'Spanish',
    'Español de España': 'Spanish',
    'Spanisch - Spanien': 'Spanish',
    'Spanyolországi spanyol': 'Spanish',
    'испанский — Латинская Америка': 'Spanish',
    'Espagnol - Espagne': 'Spanish',
    '西班牙语 - 拉丁美洲': 'Spanish',
    '西班牙语 - 拉丁美洲具有完全音频支持的语言': 'Spanish',
    'Bhs. Spanyol - Spanyol': 'Spanish',
    'Spagnolo - America Latina': 'Spanish',
    'іспанська (Латинська Америка)': 'Spanish',
    'Espagnol - Amérique latine': 'Spanish',
    'Spanisch - Lateinamerika': 'Spanish',
    'Ισπανικά - Ισπανία': 'Spanish',
    'Spansk - Spania': 'Spanish',
    'Tiếng Tây Ban Nha - TBN': 'Spanish',
    'Tiếng Tây Ban Nha - Mỹ Latin': 'Spanish',
    '西班牙文 - 西班牙': 'Spanish',
    'Hiszpański latynoamerykańskijęzyki z pełnym udźwiękowieniem': 'Spanish',
    'Español de Hispanoamérica': 'Spanish',
    'Latin. španělština': 'Spanish',
    '西班牙文 - 拉丁美洲': 'Spanish',
    'Evropská španělština': 'Spanish',

    # --- Portuguese ---
    'Portuguese': 'Portuguese',
    'Portugués': 'Portuguese',
    'Portugués - Portugal': 'Portuguese',
    'Portugués - Brasil': 'Portuguese',
    'Portuguese - Portugal': 'Portuguese',
    'Portuguese - Brazil': 'Portuguese',
    'Português (Portugal)': 'Portuguese',
    'Português (Brasil)': 'Portuguese',
    'Portugalski': 'Portuguese',
    'Portugalski brazylijski': 'Portuguese',
    'Portugués de Brasil': 'Portuguese',
    'португальский — Бразилия': 'Portuguese',
    'Portugués - Brasilidiomas': 'Portuguese',
    'Brazíliai portugál': 'Portuguese',
    'Brasilianisches Portugiesisch': 'Portuguese',
    'Bhs. Portugis - Brasil': 'Portuguese',
    'Portoghese - Brasile': 'Portuguese',
    'португальский — Португалия': 'Portuguese',
    'португальська (Бразилія)': 'Portuguese',
    'Portugais du Brésil': 'Portuguese',
    'Πορτογαλικά - Βραζιλία': 'Portuguese',
    'Portoghese - Portogallo': 'Portuguese',
    'Portugiesisch - Portugal': 'Portuguese',
    'Tiếng Bồ Đào Nha - Brazil': 'Portuguese',
    'Brazilská portugalština': 'Portuguese',
    '葡萄牙语 - 葡萄牙': 'Portuguese',
    '葡萄牙文 - 巴西具備完整音效支援的語言': 'Portuguese',
    '葡萄牙文 - 巴西': 'Portuguese',
    'Portugués de Portugal': 'Portuguese',
    'Portugais du Portugal': 'Portuguese',
    'Portugués de Brasilidiomas': 'Portuguese',

    # --- Russian ---
    'Russian': 'Russian',
    'Ruso': 'Russian',
    'Russo': 'Russian',
    'Rosyjski': 'Russian',
    'русский': 'Russian',
    'російська': 'Russian',
    '俄语': 'Russian',
    'Orosz': 'Russian',
    'Russisch': 'Russian',
    'Russe': 'Russian',
    '俄语具有完全音频支持的语言': 'Russian',
    'Bhs. Rusia': 'Russian',
    'Rosyjskijęzyki z pełnym udźwiękowieniem': 'Russian',
    'Ρωσικά': 'Russian',
    'Russiskspråk med full lydstøtte': 'Russian',
    'Tiếng Nga': 'Russian',
    'Ruština': 'Russian',
    '俄文': 'Russian',
    '俄文具備完整音效支援的語言': 'Russian',

    # --- Japanese ---
    'Japanese': 'Japanese',
    'Japonés': 'Japanese',
    'Japonês': 'Japanese',
    'Japoński': 'Japanese',
    'Japanese ()': 'Japanese',
    'японский': 'Japanese',
    '日语': 'Japanese',
    'Japonésidiomas': 'Japanese',
    'Japanisch': 'Japanese',
    'Japán': 'Japanese',
    'Japonais': 'Japanese',
    'Bhs. Jepang': 'Japanese',
    'Giapponese': 'Japanese',
    'японська': 'Japanese',
    '日语具有完全音频支持的语言': 'Japanese',
    'Ιαπωνικά': 'Japanese',
    'японська цими мовами': 'Japanese',
    '日本語フル音声対応言語': 'Japanese',
    'Tiếng Nhật': 'Japanese',
    '日文': 'Japanese',
    'Japonština': 'Japanese',
    '日本語': 'Japanese',

    # --- Korean ---
    'Korean': 'Korean',
    'Coreano': 'Korean',
    'Koreanisch': 'Korean',
    'корейский': 'Korean',
    'корейська': 'Korean',
    '韩语': 'Korean',
    'Coreanoidiomas': 'Korean',
    'Coréen': 'Korean',
    'Bhs. Korea': 'Korean',
    'Κορεατικά': 'Korean',
    'Tiếng Hàn': 'Korean',
    '韓文': 'Korean',
    'Korejština': 'Korean',
    '韩语具有完全音频支持的语言': 'Korean',

    # --- Simplified Chinese ---
    'Simplified Chinese': 'Simplified Chinese',
    'Chino simplificado': 'Simplified Chinese',
    'Chinês simplificado': 'Simplified Chinese',
    'Chiński uproszczony': 'Simplified Chinese',
    'китайский (упр.)': 'Simplified Chinese',
    '简体中文': 'Simplified Chinese',
    'китайська (спрощена)': 'Simplified Chinese',
    'Bhs. Tionghoa Sederhana': 'Simplified Chinese',
    'Cinese semplificato': 'Simplified Chinese',
    'Chinois simplifié': 'Simplified Chinese',
    'Chinesisch (vereinfacht)': 'Simplified Chinese',
    'Chino simplificadoidiomas': 'Simplified Chinese',
    'Tiếng Trung giản thể': 'Simplified Chinese',
    '簡體中文': 'Simplified Chinese',
    'Zjednodušená čínštinajazyky s plnou zvukovou podporou': 'Simplified Chinese',
    '简体中文具有完全音频支持的语言': 'Simplified Chinese',
    'Zjednodušená čínština': 'Simplified Chinese',

    # --- Polish ---
    'Polaco': 'Polish',
    'Polish': 'Polish',
    'Polonês': 'Polish',
    'Polski': 'Polish',
    'польский': 'Polish',
    'польська': 'Polish',
    '波兰语': 'Polish',
    'Lengyel': 'Polish',
    'Polnisch': 'Polish',
    'Bhs. Polandia': 'Polish',
    'Polonais': 'Polish',
    'Polacco': 'Polish',
    'Tiếng Ba Lan': 'Polish',
    'Polština': 'Polish',
    '波蘭文': 'Polish'
}

dictionnaire_langues = {
    "English": [
        'English', 'Inglés', 'Inglês', 'Angielski', 'английский', 'Inglese', 
        'англійська', '英语', 'Inglésidiomas', 'Englisch', 'Angol', 'Anglais', 
        'Bhs. Inggris', 'Αγγλικά', 'Engelsk', '英語', 'Tiếng Anh', '英文', 
        'Angielskijęzyki z pełnym udźwiękowieniem', 'Angličtina'
    ],
    "French": [
        'French', 'Francés', 'Francês', 'Francuski', '#lang_français', 
        'французский', 'Francese', 'французька', '法语', 'Französisch', 
        'Francia', 'Français', 'Bhs. Prancis', 'Γαλλικά', 'Fransk', 
        'Tiếng Pháp', '法文', 'Francouzština'
    ],
    "German": [
        'German', 'Alemán', 'Alemão', 'Niemiecki', 'немецкий', 'Tedesco', 
        'German;', 'німецька', '德语', 'Deutsch', 'Német', 'Allemand', 
        'Bhs. Jerman', 'Γερμανικά', 'Tysk', 'Tiếng Đức', '德文', 'Němčina'
    ],
    "Spanish": [
        'Spanish - Spain', 'Español - España', 'Español - Latinoamérica', 
        'Spanish - Latin America', 'Espanhol (Espanha)', 'Espanhol (América Latina)', 
        'Hiszpański', 'Hiszpański latynoamerykański', 'испанский — Испания', 
        'Spagnolo - Spagna', 'іспанська (Іспанія)', '西班牙语 - 西班牙', 
        'Español de España', 'Spanisch - Spanien', 'Spanyolországi spanyol', 
        'испанский — Латинская Америка', 'Espagnol - Espagne', '西班牙语 - 拉丁美洲', 
        '西班牙语 - 拉丁美洲具有完全音频支持的语言', 'Bhs. Spanyol - Spanyol', 
        'Spagnolo - America Latina', 'іспанська (Латинська Америка)', 
        'Espagnol - Amérique latine', 'Spanisch - Lateinamerika', 
        'Ισπανικά - Ισπανία', 'Spansk - Spania', 'Tiếng Tây Ban Nha - TBN', 
        'Tiếng Tây Ban Nha - Mỹ Latin', '西班牙文 - 西班牙', 
        'Hiszpański latynoamerykańskijęzyki z pełnym udźwiękowieniem', 
        'Español de Hispanoamérica', 'Latin. španělština', 
        '西班牙文 - 拉丁美洲', 'Evropská španělština'
    ],
    "Portuguese": [
        'Portugués - Portugal', 'Portugués - Brasil', 'Portuguese - Portugal', 
        'Portuguese - Brazil', 'Português (Portugal)', 'Português (Brasil)', 
        'Portugalski', 'Portugalski brazylijski', 'Portugués de Brasil', 
        'португальский — Бразилия', 'Portugués - Brasilidiomas', 'Brazíliai portugál', 
        'Brasilianisches Portugiesisch', 'Bhs. Portugis - Brasil', 
        'Portoghese - Brasile', 'португальский — Португалия', 
        'португальська (Бразилія)', 'Portugais du Brésil', 
        'Πορτογαλικά - Βραζιλία', 'Portoghese - Portogallo', 
        'Portugiesisch - Portugal', 'Tiếng Bồ Đào Nha - Brazil', 
        'Brazilská portugalština', '葡萄牙语 - 葡萄牙', 
        '葡萄牙文 - 巴西具備完整音效支援的語言', '葡萄牙文 - 巴西', 
        'Portugués de Portugal', 'Portugais du Portugal',
        'Portugués de Brasilidiomas'
    ],
    "Russian": [
        'Russian', 'Ruso', 'Russo', 'Rosyjski', 'русский', 'російська', '俄语', 
        'Orosz', 'Russisch', 'Russe', '俄语具有完全音频支持的语言', 'Bhs. Rusia', 
        'Rosyjskijęzyki z pełnym udźwiękowieniem', 'Ρωσικά', 
        'Russiskspråk med full lydstøtte', 'Tiếng Nga', 'Ruština', '俄文', 
        '俄文具備完整音效支援的語言'
    ],
    "Japanese": [
        'Japanese', 'Japonés', 'Japonês', 'Japoński', 'Japanese ()', 'японский', 
        '日语', 'Japonésidiomas', 'Japanisch', 'Japán', 'Japonais', 'Bhs. Jepang', 
        'Giapponese', 'японська', '日语具有完全音频支持的语言', 'Ιαπωνικά', 
        'японська цими мовами', '日本語フル音声対応言語', 'Tiếng Nhật', '日文', 
        'Japonština', '日本語'
    ],
    "Korean": [
        'Korean', 'Coreano', 'Koreanisch', 'корейский', 'корейська', '韩语', 
        'Coreanoidiomas', 'Coréen', 'Bhs. Korea', 'Κορεατικά', 'Tiếng Hàn', 
        '韓文', 'Korejština', '韩语具有完全音频支持的语言'
    ],
    "Simplified Chinese": [
        'Simplified Chinese', 'Chino simplificado', 'Chinês simplificado', 
        'Chiński uproszczony', 'китайский (упр.)', '简体中文', 
        'китайська (спрощена)', 'Bhs. Tionghoa Sederhana', 'Cinese semplificato', 
        'Chinois simplifié', 'Chinesisch (vereinfacht)', 
        'Chino simplificadoidiomas', 'Tiếng Trung giản thể', '簡體中文', 
        'Zjednodušená čínštinajazyky s plnou zvukovou podporou', 
        '简体中文具有完全音频支持的语言', 'Zjednodušená čínština'
    ],
    "Polish": [
        'Polaco', 'Polish', 'Polonês', 'Polski', 'польский', 'польська', 
        '波兰语', 'Lengyel', 'Polnisch', 'Bhs. Polandia', 'Polonais', 'Polacco', 
        'Tiếng Ba Lan', 'Polština', '波蘭文'
    ]
}




def strip_all_bad_characters_from_lang(raw_text):
    return raw_text.replace('<strong>', '').replace('</strong>', '').replace('<br>', ' ').replace('*', '')   \
                   .replace('\r', ' ').replace('\n', ' ').replace('[b]', '').replace('[/b]', '')  \
                   .replace('languages with full audio support', '')    \
                   .replace('all with full audio support', '')  \
                   .replace('(all with full audio support)', '')  \
                   .replace('(full audio)', '')   \
                   .replace('con supporto audio complet', '')    \
                   .replace('Langues avec support audio complet', '')   \
                   .replace('con localización de audio', '')    \
                   .replace('Sprachen mit voller Audiounterstützung', '')    \
                   .replace('Cekobahasa dengan dukungan audio penuh', '')   \
                   .replace('idiomas com suporte total de áudio', '')   \
                   .replace('озвучивание доступно на этих языках', '')   \
                   .replace('повністю озвучено', '')   \
                   .strip()


def check_full_audio_support(raw_text):
    AUDIO_FULL_PATTERNS = [
        'languages with full audio support',
        'all with full audio support',
        '(all with full audio support)',
        '(full audio)',
        'con supporto audio complet',
        'langues avec support audio complet',
        'con localización de audio',
        'sprachen mit voller audiounterstützung',
        'cekobahasa dengan dukungan audio penuh',
        'idiomas com suporte total de áudio',
        'озвучивание доступно на этих языках',
        'повністю озвучено',
    ]

    raw_text_lower = raw_text.lower()

    if any(pattern in raw_text_lower for pattern in AUDIO_FULL_PATTERNS):
        return True
    else:
        return False

In [156]:
# supported_languages AUDIO SUPPORT

def extract_audio_info(raw_text):
    if pd.isna(raw_text):
        return 0, 0
    
    # On découpe par langue (séparateur virgule)
    # Ex: "English*, French, German*" -> ["English*", " French", " German*"]
    langs = raw_text.split(',')
    #print(langs)
    # 1. Compter combien de segments contiennent "*"
    audio_count = sum(1 for l in langs if '*' in l)
    if check_full_audio_support(raw_text):
      audio_count = raw_text.count(',')
      # We suppose at least 1 audio language
    if audio_count == 0:
        audio_count = 1

    stripped_langs = [ strip_all_bad_characters_from_lang(lang) for lang in langs ]
    #print(stripped_langs)
    # 2. Vérifier si l'Anglais a l'audio
    #has_english_audio = 0
    #for lang in stripped_langs:
    #   for lang_translation in dictionnaire_langues['English']:
    #      if lang_translation in lang and ('*' in lang or check_full_audio_support(raw_text)):
    #         has_english_audio = 1


    has_french_audio = 0
    for lang in stripped_langs:
       for lang_translation in dictionnaire_langues['French']:
          if lang_translation in lang and ('*' in lang or check_full_audio_support(raw_text)):
             has_french_audio = 1
    
    return audio_count, has_french_audio

# Application
dataset[['num_audio_languages', "french_audio"]] = dataset['supported_languages'].apply(
    lambda x: pd.Series(extract_audio_info(x))
)

# Vérification rapide
print(dataset[['name', 'num_audio_languages', "french_audio"]].head())

                        name  num_audio_languages  french_audio
0             Counter-Strike                    7             1
1      Team Fortress Classic                    1             0
2              Day of Defeat                    1             0
3         Deathmatch Classic                    1             0
4  Half-Life: Opposing Force                    1             0


In [157]:
print( len( dataset[dataset['num_audio_languages'] == 0] ) )

0


In [158]:
# English, French, German, Italian, Spanish - Spain, Japanese (all with full audio support)
def get_all_distinct_languages(raw_text, all_langs, not_trans_langs):
    if pd.isna(raw_text):
        return 0, 0
    langs = raw_text.split('<br>')[0]
    #if "all with full audio support" in raw_text:
    #    print(raw_text)

    langs = raw_text.split(',')
    for lang in langs:
        lang = strip_all_bad_characters_from_lang(lang)
        if lang not in lookup_langues.keys():
            if lang not in not_trans_langs:
                not_trans_langs.append(lang)
        else:
            if lookup_langues[lang] not in all_langs:
                all_langs.append(lookup_langues[lang])


all_langs = []
not_trans_langs = []

for index, row in dataset.iterrows():
    get_all_distinct_languages(row['supported_languages'], all_langs, not_trans_langs)

print("All languages registered languages", all_langs)
print("Languages not in the lookup dictionnary (either not one of top 10 languages or translation not in the lookup dictionnary", not_trans_langs, "ajajaj")


All languages registered languages ['English', 'French', 'German', 'Spanish', 'Simplified Chinese', 'Korean', 'Russian', 'Japanese', 'Polish', 'Portuguese']
Languages not in the lookup dictionnary (either not one of top 10 languages or translation not in the lookup dictionnary ['Italian', 'Traditional Chinese', 'Italiano', 'Chino tradicional', 'Holandés', 'Danés', 'Finés', 'Noruego', 'Sueco', 'Tailandés', 'Búlgaro', 'Checo', 'Griego', 'Húngaro', 'Rumano', 'Turco', 'Ucraniano idiomas', 'Thai', 'Turkish', 'Dutch', 'Danish', 'Finnish', 'Norwegian', 'Swedish', 'Bulgarian', 'Czech', 'Greek', 'Hungarian', 'Romanian', 'Ukrainian', 'Vietnamese', 'Dinamarquês', 'Holandês', 'Finlandês', 'Norueguês', 'Chinês tradicional', 'Tcheco', 'Grego', 'Romeno', 'Tailandês', 'Ucraniano', 'Vietnamita', 'Czeski', 'Duński', 'Holenderski', 'Fiński', 'Węgierski', 'Włoski', 'Koreański', 'Norweski', 'Rumuński', 'Szwedzki', 'Tajski', 'Chiński tradycyjny', 'Turecki', 'Bułgarski', 'Ukraiński', 'Grecki', 'Wietnamski', 

In [159]:
# Je reprends ta liste cible définie précédemment
target_languages = [
    'English', 'French', 'German', 'Spanish', 'Portuguese', 
    'Russian', 'Japanese', 'Korean', 'Simplified Chinese', 'Polish'
]

def process_languages_with_dict_robust(text):
    # 1. Gestion des vides
    if pd.isna(text) or text == "":
        return pd.Series([0] + [0]*len(target_languages))
    
    # 2. Nettoyage minimal
    # On garde les espaces car "Spanish - Spain" en a besoin
    # On retire juste le HTML et les indicateurs audio
    clean_text = strip_all_bad_characters_from_lang(text)
    
    # 3. SCANNING (C'est ici que tout change)
    normalized_set = set()
    
    # Au lieu de couper par virgule, on itère sur TON dictionnaire
    # On vérifie si la clé (ex: "Inglés") existe dans le texte global
    for dirty_key, clean_val in lookup_langues.items():
        if dirty_key in clean_text:
            normalized_set.add(clean_val)
            
    # 4. Calcul des Features (Identique à avant)
    num_langs = len(normalized_set)
    
    one_hot_results = []
    for target in target_languages:
        if target in normalized_set:
            one_hot_results.append(1)
        else:
            one_hot_results.append(0)
            
    return pd.Series([num_langs] + one_hot_results)

# --- APPLICATION ---
new_cols = ['num_languages'] + [f'Lang_{l}' for l in target_languages]

# Attention : Si ton dataset est très gros, cette méthode est un peu plus lente 
# que le split, mais elle est infiniment plus précise pour tes données sales.
dataset[new_cols] = dataset['supported_languages'].apply(process_languages_with_dict_robust)

dataset = dataset.drop(columns=['supported_languages'])

In [160]:
# --- VÉRIFICATION ---
# On vérifie si ton problème de 3032 jeux sans anglais a diminué
missing_english = len(dataset[dataset['Lang_English'] == 0])
print(f"Nombre de jeux sans Anglais détectés : {missing_english}")

# Testons ton cas spécifique pour être sûr
test_string = "RussianEnglishSpanish - SpainFrenchJapaneseCzech"
res_test = process_languages_with_dict_robust(test_string)
print("-" * 30)
print(f"Test sur la chaîne sale : 'English' détecté ? {res_test.iloc[1] == 1}") # Index 1 = Lang_English
print(f"Nombre de langues trouvées dans la chaîne sale : {res_test.iloc[0]}")

Nombre de jeux sans Anglais détectés : 3029
------------------------------
Test sur la chaîne sale : 'English' détecté ? True
Nombre de langues trouvées dans la chaîne sale : 5


In [161]:
dataset.head()

,name,appid,required_age,is_free,controller_support,short_description,initialprice,is_hit,windows,mac,...,Lang_English,Lang_French,Lang_German,Lang_Spanish,Lang_Portuguese,Lang_Russian,Lang_Japanese,Lang_Korean,Lang_Simplified Chinese,Lang_Polish
0,Counter-Strike,10,0,0,0,Play the world's number 1 online action game. ...,9.99,1,True,True,...,1,1,1,1,0,0,0,1,1,0
1,Team Fortress Classic,20,0,0,0,One of the most popular online action games of...,4.99,1,True,True,...,1,1,1,1,0,1,0,1,1,0
2,Day of Defeat,30,0,0,0,Enlist in an intense brand of Axis vs. Allied ...,4.99,1,True,True,...,1,1,1,1,0,0,0,0,0,0
3,Deathmatch Classic,40,0,0,0,Enjoy fast-paced multiplayer gaming with Death...,4.99,1,True,True,...,1,1,1,1,0,1,0,1,1,0
4,Half-Life: Opposing Force,50,0,0,0,Return to the Black Mesa Research Facility as ...,4.99,1,True,True,...,1,1,1,0,0,0,0,1,0,0


In [162]:
dataset.columns

Index(['name', 'appid', 'required_age', 'is_free', 'controller_support',
       'short_description', 'initialprice', 'is_hit', 'windows', 'mac',
       'linux', 'genre_Accounting', 'genre_Action', 'genre_Adventure',
       'genre_Animation & Modeling', 'genre_Audio Production', 'genre_Casual',
       'genre_Design & Illustration', 'genre_Early Access', 'genre_Education',
       'genre_Free to Play', 'genre_Game Development', 'genre_Gore',
       'genre_Indie', 'genre_Massively Multiplayer', 'genre_Movie',
       'genre_Nudity', 'genre_Photo Editing', 'genre_RPG', 'genre_Racing',
       'genre_Sexual Content', 'genre_Short', 'genre_Simulation',
       'genre_Software', 'genre_Software Training', 'genre_Sports',
       'genre_Strategy', 'genre_Utilities', 'genre_Video Production',
       'genre_Violent', 'genre_Web Publishing', 'categorie_Captions/Subtitles',
       'categorie_Co-op', 'categorie_Cross-Platform Multiplayer',
       'categorie_Family Sharing', 'categorie_Full Controller Su

In [163]:
# Export du dataset propre
dataset.to_csv('steam_dataset_clean.csv', index=False, encoding='utf-8-sig')

print("Export terminé avec succès !")

Export terminé avec succès !
